In [ ]:
pip install pandas pyarrow

In [1]:
import json
import pandas as pd
import gzip
import ast
from datetime import datetime

In [ ]:
#archivo: "C:\Users\Usuario\Henry\PI01MLOPs\DataSet\Users_items\users_items.json.gz"

In [2]:
# Desarrollo función leer_Json_gzip() que lee los archivos 'comprimidos.json.gz` y devuelve una lista de diccionarios Python.
def leer_json_gzip(users_items):

    # Abrir el archivo users_items.json.gz en modo de lectura y decodificamos el contenido del archivo.
    with gzip.open(users_items, 'rt', encoding='utf-8') as miArchivo:

        # Recorremos el contenido del archivo y evaluamos cada línea como una expresión Python.
        # El resultado de la evaluación es un diccionario Python.
        # La función strip() elimina los espacios en blanco al principio y al final de la línea.
        return [ast.literal_eval(line.strip()) for line in miArchivo]
    #Usamos ast.literal_eval para convertir cada linea en un diccionario y lo agregamos a la lista

In [3]:
datos_users_items = []
for i  in gzip.open('users_items.json.gz'):
    datos_users_items.append(ast.literal_eval(i.decode('utf-8')))

DataFrame_users_items = pd.DataFrame(datos_users_items)



In [ ]:
DataFrame_users_items

In [5]:
DataFrame_users_items.to_parquet('users_items.parquet', engine='pyarrow')

In [ ]:
DataFrame_users_items.head()

In [7]:
#Revisé el tipo de datos que hay en cada columna
tipo_de_datos_columna_items_count = DataFrame_users_items['items'].dtype
print("Tipo de datos de la columna 'items':", tipo_de_datos_columna_items_count)

Tipo de datos de la columna 'items': object


In [8]:
# Dado que el archivo esta muy largo no desanidaré la columna hasta que el enunciado no lo indique y queda asi el archivo que ya está manejable
# Al hacer la función veo que debo desanidar columna "items" para encontrar el tiempo jugado

DataFrame_users_items_exploded = DataFrame_users_items.explode('items')


In [9]:
# Normalizo los datos y creo un nuevo data frame desanidado
DataFrame_users_items_desanidado = pd.json_normalize(DataFrame_users_items_exploded['items'].dropna())

In [10]:
# Volver a indexar
DataFrame_users_items_desanidado.reset_index(inplace=True)
DataFrame_users_items_exploded.reset_index(inplace=True)

In [11]:
#  Concateno Data Frames y borro columna original items
user_items_final = pd.concat([DataFrame_users_items_exploded, DataFrame_users_items_desanidado], axis=1)
df_user_item_final =  user_items_final.drop(columns=['items'])

In [13]:

#vuelvo a hacer datframe con los datos del archivo steam_games
# le pego el data frame de steam_games, le qiuto nulos otra vez, asi que traje el dataFrame del archico users:items

df_steam_games = pd.read_json("steam_games.json.gz" , lines=True)
df_steam_games_sinnulos = df_steam_games.dropna()
DataFrame_users_items_desanidado = pd.json_normalize(DataFrame_users_items_exploded['items'].dropna())

In [ ]:
DataFrame_users_items_desanidado

In [15]:
# Transformo el año que resultó del explode a numero para que se le quite el punto

df_steam_games_sinnulos_Exploded = df_steam_games_sinnulos.explode('tags')
df_steam_games_sinnulos_Exploded['release_date'] = pd.to_datetime(df_steam_games_sinnulos_Exploded['release_date'], errors='coerce')
df_steam_games_sinnulos_Exploded['release_year'] = df_steam_games_sinnulos_Exploded['release_date'].dt.year


Trato de vincular  las etiquetas de columna df_steam_games_sinnulos_Exploded con la columna DataFrame_users_items_desanidado item_name usando  Pandas.

Me doy cuenta que el contenido de la columna app_name en streams es el mismo que item_name en user_items así que a travez de esa columna hago el merge

Entonces creo la fusion usando merge así: merged_df = df1.merge(df2, on='app_name', how='inner')

En este caso tenemos dos Data Frames  de datos que se entrelazan con dos nombres de columnas diferentes. 

merged_df = df1.merge(df2, left_on='titapp_namele', right_on='item', how='inner'), donde 'app_name' es una columna en df1 y el item_name está en el dataframe df2.

entonces creo  un nuevo data frame que une 2 data frames  usando dos nombres de campo:

In [16]:

merged_df = df_steam_games_sinnulos_Exploded.merge(DataFrame_users_items_desanidado, left_on='app_name', right_on='item_name', how='inner')


In [ ]:
merged_df

In [18]:
# aca defino la función según la consigna, en este caso es la función1

def PlayTimeGenre(merged_df, genre):

    # Filtrar el DataFrame para incluir solo filas con el género especificado con tags que tiene la misma información de genres

    genre_df = merged_df[merged_df['tags'] == genre]

    if genre_df.empty:
        return "No hay datos para este genero."

    # Se agrupa el DataFrame filtrado por 'año_lanzamiento' y busque el año con el tiempo de reproducción máximo

    max_playtime_year = genre_df.groupby('release_year')['playtime_forever'].sum().idxmax()

    return max_playtime_year

In [17]:
# Teniendo todos los datos en la misma tabla se pueden hacer las consultas necesarias

In [19]:
genres = "Free to Play"  # Se ppuede reemplazar con el género que se desea consultar
year_with_max_playtime = PlayTimeGenre(merged_df, genres)
print(f"El año con la mayor cantidad de horas jugadas de {genres} es {year_with_max_playtime}.")


El año con la mayor cantidad de horas jugadas de Free to Play es 2013.0.
